In [19]:
import json
from pathlib import Path

import pandas as pd
from datasets import load_dataset

from asr.comparison import MultipleTextsAlignment

dataset = load_dataset('dangrebenkin/long_audio_youtube_lectures')['train']
name_to_transcription = dict(zip(dataset['name'], dataset['transcription']))

results_list = []
for filepath in Path('/home/oleg/pisets_test_results').glob('*.json'):
    data = json.loads(filepath.read_text())
    for pipeline_name, transcription in data['transcriptions'].items():
        results_list.append({
            'audio_name': data['audio_name'],
            'pipeline_name': pipeline_name,
            'transcription': transcription,
        })

results = pd.DataFrame(results_list)

results['alignment'] = results.apply(
    lambda row: MultipleTextsAlignment.from_strings(
        name_to_transcription[row['audio_name']],
        row['transcription']
    ),
    axis='columns'
)
results['wer'] = results['alignment'].apply(lambda al: al.wer()['wer'])

In [20]:
results.pivot_table(values='wer', index='pipeline_name', columns='audio_name')

audio_name,harvard,kolodezev,lankov,savvateev,tuberculosis,zaliznyak,zhirinovsky
pipeline_name,,,,,,,
Baseline Whisper longform,0.010929,NaN,NaN,NaN,NaN,0.158086,NaN
Baseline Whisper pipeline,0.045537,0.155228,0.147354,0.192440,0.199501,0.131617,0.115655
Pisets WhisperV3 (segments 1s-20s),0.015938,0.129357,0.087513,0.216986,NaN,0.116751,0.060306
Pisets WhisperV3 stretched (segments 1s-20s),0.037796,0.114984,0.109997,0.348061,NaN,0.139231,0.072697
W2V2 Golos LM,0.149818,0.271649,0.316845,0.629357,NaN,0.250544,0.261875
